In [1]:
import threading
import itertools
import time
import sys

class Signal:
    go = True
    

def spin(msg, signal):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        # 使用退格符（\x08）把光标移回来
        write('\x08' * len(status))
        time.sleep(.1)
        if not signal.go:
            break
    write(' ' * len(status) + '\x08' * len(status))
    
def slow_function():
    # 假装等待I/O一段时间
    time.sleep(3)
    return 42

def supervisor(): #9
    signal = Signal()
    spinner = threading.Thread(target=spin, args=('thinking!', signal))
    print('spinner object:', spinner) #10
    spinner.start() #11
    # 运行slow_function阻塞主线程
    result = slow_function() #12
    signal.go = False #13
    # 等待spinner线程结束
    spinner.join() #14
    return result

def main():
    result = supervisor() #15
    print('Answer:', result)
    
if __name__ == '__main__':
    main()

spinner object: <Thread(Thread-6, initial)>
| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking          Answer: 42


In [1]:
import asyncio
import itertools
import sys


@asyncio.coroutine #1
def spin(msg): #2
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status))
        try:
            yield from asyncio.sleep(.1) #3
        except asyncio.CancelledError: #4
            break
    write(' ' * len(status) + '\x08' * len(status))
    

@asyncio.coroutine
def slow_function(): #5
    # 假装等待I/O一段时间
    yield from asyncio.sleep(3) #6
    return 42

@asyncio.coroutine
def supervisor(): #7
    spinner = asyncio.async(spin('thinking!')) #8
    print('spinner object:', spinner) #9
    result = yield from slow_function() #10
    spinner.cancel() #11
    return result

def main():
    loop = asyncio.get_event_loop() #12
    result = loop.run_until_complete(supervisor()) #13
    loop.close()
    print('Answer:', result)
    

if __name__ == '__main__':
    main()

SyntaxError: invalid syntax (<ipython-input-1-3c445077ad79>, line 29)